In [1]:
import json
import altair as alt
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, isdir, join, splitext

In [2]:
input_directory = "../pareto"
output_drectory = "./"

In [3]:
graphs = [f for f in listdir(input_directory) if isfile(join(input_directory, f))]

In [4]:
reorder_algorithms = ["hub", "rand", "gorder", "RCM", "write_reorder2", "deg"]
# reorder_algorithms = ["hub", "gorder", "RCM", "write_reorder2", "deg"]

In [5]:
pretty_reorder_algo = {'hub': 'hub', 'deg': 'deg', 'RCM': 'RCM',
                        'rand': 'rand', 'write_reorder2': 'BOBA', 'gorder': 'Gorder'}

In [6]:
results = pd.DataFrame(columns=['Dataset', 'Reorder algorithm', 'graph_algorithm', 'Graph algorithm runtime', 'Reorder time'])

In [7]:
graph_names_ = ['coAuthorsCiteseer', 'coAuthorsDBLP', 'delaunay_n22', 'delaunay_n23', 'delaunay_n24'
                , 'great-britain_osm', 'hollywood-2009', 'rgg_n_2_22_s0', 'rgg_n_2_23_s0', 'rgg_n_2_24_s0'
                , 'roadNet-CA', 'road_usa', 'soc-LiveJournal1', 'ljournal-2008', 'arabic-2005']
ignore_graphs_ = ['delaunay_n22', 'hollywood-2009']

def get_graph_name(graph):
        for g in graph_names_:
                if g in graph or graph in g:
                        return g

def is_ignored_graph_name(graph):
        for g in ignore_graphs_:
                if g in graph or graph in g:
                        return True
        return False


In [8]:
def get_graph_algorithm_name(graph):
        temp = ['tc', 'sssp', 'spmv', 'pr']
        for t in temp:
                if t in graph or graph in t:
                        return t

In [9]:
def get_reorder_algorithms(graph):
    for reorder_algo in reorder_algorithms:
        if graph.lower() in reorder_algo.lower() or reorder_algo.lower() in graph.lower():
            return reorder_algo
    print(graph)

In [10]:
def get_reoder_time(input_file, reorder_algo):
    file = open(input_file)
    loaded_json = json.load(file)
    # print(loaded_json)
    reorder_times = {}
    for graph in graph_names_:
        if is_ignored_graph_name(graph):
            continue
        else:
            reorder_times[graph] = {}
    for row in loaded_json:
        graph_name = get_graph_name(row['graph_name'])
        reoder_algo = pretty_reorder_algo[get_reorder_algorithms(row['graph_name'])]
        # results[graph_name][]
        reorder_times[graph_name][reoder_algo] = float(row['Reorder_Time'])
    return reorder_times

In [11]:
reorder_times = get_reoder_time(input_directory + '/reorder-time/reorder.json', reorder_algorithms)
# get_reoder_time(input_directory + '/reorder-time/test.json', reorder_algorithms)


In [12]:
for graph in graphs:
    file = open(input_directory + '/' + graph)
    try: 
        loaded_json = json.load(file)
        algo_time = float(loaded_json['run-time'])
        for reorder_algo in reorder_algorithms:
            if reorder_algo in graph:
                graph_name = get_graph_name(graph)
                if(is_ignored_graph_name(graph_name)):
                    continue
                reorder_algo_name = pretty_reorder_algo[reorder_algo]
                graph_algo_name = get_graph_algorithm_name(graph)
                reorder_time = float(0.1)
                if 'rand' in graph:
                    reorder_time = float(0.1)
                else:
                    try:
                        reorder_time = float(reorder_times[graph_name][reorder_algo_name])
                    except: 
                        print (graph_name)
                        print (reorder_algo_name)
                        

                    
                # print(graph_name)
                # print(reorder_algo_name)
                results.loc[len(results.index)] = [graph_name, reorder_algo_name, graph_algo_name,
                                                    algo_time, reorder_time]
    except:
        print("Failed to open: ", graph)


In [13]:
def make_pareto(algo_name):
    to_plot = results[results['graph_algorithm'] == algo_name.lower()]
    # to_plot = to_plot[results['reorder_algorithm'] != 'rand']

    chart = alt.Chart(to_plot).mark_point(filled=True).encode(
        alt.Y('Reorder time', scale=alt.Scale(type='log')), 
        alt.X('Graph algorithm runtime', scale=alt.Scale(type='log')), 
        color=alt.Color('Reorder algorithm'),
        shape=alt.Shape('Dataset')).properties(
        title=algo_name)
    chart.save(algo_name + '.svg')
    return chart

In [14]:
make_pareto('PR')

alt.Chart(...)

In [15]:
make_pareto('TC')

alt.Chart(...)

In [16]:
make_pareto('SSSP')

alt.Chart(...)

In [17]:
make_pareto('SpMV')

alt.Chart(...)